In [1]:
import subprocess

def run_sim(simTime, appPeriod, nDevices, dataUpType, realisticModel, endDeviceType, gwDc):
    ns3_script = "lora-device-classes-example"

    cmd_params = " --gwDc="+f"{gwDc}"+" --endDeviceType="+f"{endDeviceType}"+" --appPeriod="+f"{appPeriod}"+" --simulationTime="+f"{simTime}"+" --nDevices="+f"{nDevices}"+" --dataUpType="+f"{dataUpType}"+" --realisticModel="+f"{realisticModel}"

    command = "/home/adi/Workspace/ns-allinone-3.37/ns-3.37/ns3 run "+ f'"{ns3_script + cmd_params}"'
    print(command)
    result=""
    try:
        result = subprocess.run(command, shell=True, check=True, stdout=subprocess.PIPE, text=True).stdout.splitlines()[-1]
    except subprocess.CalledProcessError as e:
        print(f"Error running script: {e}")
    
    return result

In [2]:
import smtplib
from email.mime.text import MIMEText

def send_email(subject, message, to):
    try:
        server = smtplib.SMTP('smtp.gmail.com', 587)
        server.starttls()
        server.login('phani.jsp@gmail.com', 'rkyq vvjd xtwh kfwx')
        
        msg = MIMEText(message)
        msg['Subject'] = subject
        msg['From'] = 'vlad@Castlevania.com'
        msg['To'] = to
        
        server.sendmail('vlad@Castlevania', to, msg.as_string())
        server.quit()
        print('Successfully sent email to %s:' % (to))
    except Exception as e:
        print('Failed to send email to %s:' % (to))
        print(e)

In [3]:
import numpy as np

def devices_log_list():
    initial_value = 1
    #Lambda value 5, corresponds to almost 500 devices at current app period of 120
    final_value = 4500
    num_points = 40
    log_values = np.logspace(np.log10(initial_value), np.log10(final_value), num=num_points)
    # log_values = np.linspace(initial_value, final_value, num=num_points)
    return log_values

In [4]:
# for i  in devices_log_list():
#     print(max(1,round(i)))


In [5]:
import concurrent.futures
import json
import math
import os

def run_scenario(traffic_type, realistic_model, device_class, scenario_name, gwDc):
    
    simTime = 800
    appPeriod = 600
    data_all = []
    futures = []

    with concurrent.futures.ThreadPoolExecutor() as executor:

        deviceLogIter = devices_log_list()
        for i in deviceLogIter:
            deviceCount = max(1,round(i))
            print(scenario_name+", Simualtion with device count: "+str(deviceCount))
            future = executor.submit(run_sim, simTime, appPeriod, deviceCount, traffic_type, realistic_model, device_class, gwDc)
            futures.append(future)



    for future in futures:
        data = json.loads(future.result())
        if float(data["ULPDR"]["Total"])>0:
            data_all.append(data)
            print(str(float(data["ULPDR"]["Success"])/float(data["ULPDR"]["Total"]))+" L: "+str(float(data["ULPDR"]["Total"])/float(simTime)))
        
    test_folder_name = 'test_scenario_gw_dc_on_off'
    test_sub_folder = ''
    
    if(gwDc):
        test_sub_folder = 'on'
    else:
        test_sub_folder = 'off'

    target_folder = os.path.join(test_folder_name, test_sub_folder)
    if not os.path.exists(target_folder):
        os.makedirs(target_folder)

    filePath = os.path.join(target_folder, scenario_name)

    with open(filePath, 'w') as f:
        json.dump(data_all, f)

In [6]:
import threading

threads = []
threads.append(threading.Thread(target=run_scenario,args=(0, False, 'A', "ULPDR_UNCONFIRMED_A", True)))
threads.append(threading.Thread(target=run_scenario,args=(1, False, 'A', "ULPDR_CONFIRMED_A", True)))
threads.append(threading.Thread(target=run_scenario,args=(2, False, 'A', "ULPDR_MIXED_A", True)))

threads.append(threading.Thread(target=run_scenario,args=(0, False, 'A', "ULPDR_UNCONFIRMED_A", False)))
threads.append(threading.Thread(target=run_scenario,args=(1, False, 'A', "ULPDR_CONFIRMED_A", False)))
threads.append(threading.Thread(target=run_scenario,args=(2, False, 'A', "ULPDR_MIXED_A", False)))

In [7]:

threads.append(threading.Thread(target=run_scenario,args=(0, False, 'B', "ULPDR_UNCONFIRMED_B", True)))
threads.append(threading.Thread(target=run_scenario,args=(1, False, 'B', "ULPDR_CONFIRMED_B", True)))
threads.append(threading.Thread(target=run_scenario,args=(2, False, 'B', "ULPDR_MIXED_B", True)))

threads.append(threading.Thread(target=run_scenario,args=(0, False, 'B', "ULPDR_UNCONFIRMED_B", False)))
threads.append(threading.Thread(target=run_scenario,args=(1, False, 'B', "ULPDR_CONFIRMED_B", False)))
threads.append(threading.Thread(target=run_scenario,args=(2, False, 'B', "ULPDR_MIXED_B", False)))

In [8]:

threads.append(threading.Thread(target=run_scenario,args=(0, False, 'C', "ULPDR_UNCONFIRMED_C", True)))
threads.append(threading.Thread(target=run_scenario,args=(1, False, 'C', "ULPDR_CONFIRMED_C", True)))
threads.append(threading.Thread(target=run_scenario,args=(2, False, 'C', "ULPDR_MIXED_C", True)))

threads.append(threading.Thread(target=run_scenario,args=(0, False, 'C', "ULPDR_UNCONFIRMED_C", False)))
threads.append(threading.Thread(target=run_scenario,args=(1, False, 'C', "ULPDR_CONFIRMED_C", False)))
threads.append(threading.Thread(target=run_scenario,args=(2, False, 'C', "ULPDR_MIXED_C", False)))


In [9]:
import time
start_time = time.time()
for thread in threads:
    thread.start()

for thread in threads:
    thread.join()
end_time = time.time()

elapsed_time = end_time - start_time
print(elapsed_time)
send_email('[IMP] Sim Plot Run Status GWDC_On_Off', "Simulation complete!.\nTotal run time: {:.6f} seconds.\nPlease check the results on your vm.\n\nThank you,\nVlad".format(elapsed_time), 'aditya.jagatha@trojans.dsu.edu')

ULPDR_UNCONFIRMED_A, Simualtion with device count: 1
/home/adi/Workspace/ns-allinone-3.37/ns-3.37/ns3 run "lora-device-classes-example --gwDc=True --endDeviceType=A --appPeriod=600 --simulationTime=800 --nDevices=1 --dataUpType=0 --realisticModel=False"
ULPDR_UNCONFIRMED_A, Simualtion with device count: 1
/home/adi/Workspace/ns-allinone-3.37/ns-3.37/ns3 run "lora-device-classes-example --gwDc=True --endDeviceType=A --appPeriod=600 --simulationTime=800 --nDevices=1 --dataUpType=0 --realisticModel=False"
ULPDR_UNCONFIRMED_A, Simualtion with device count: 2
/home/adi/Workspace/ns-allinone-3.37/ns-3.37/ns3 run "lora-device-classes-example --gwDc=True --endDeviceType=A --appPeriod=600 --simulationTime=800 --nDevices=2 --dataUpType=0 --realisticModel=False"
ULPDR_UNCONFIRMED_A, Simualtion with device count: 2
ULPDR_CONFIRMED_A, Simualtion with device count: 1
/home/adi/Workspace/ns-allinone-3.37/ns-3.37/ns3 run "lora-device-classes-example --gwDc=True --endDeviceType=A --appPeriod=600 --simu

+0.000000000s Completed configuration
+0.000000000s Running simulation...
Computing performance metrics...


/home/adi/Workspace/ns-allinone-3.37/ns-3.37/ns3 run "lora-device-classes-example --gwDc=True --endDeviceType=B --appPeriod=600 --simulationTime=800 --nDevices=1 --dataUpType=2 --realisticModel=False"ULPDR_MIXED_B, Simualtion with device count: 2
/home/adi/Workspace/ns-allinone-3.37/ns-3.37/ns3 run "lora-device-classes-example --gwDc=True --endDeviceType=B --appPeriod=600 --simulationTime=800 --nDevices=2 --dataUpType=2 --realisticModel=False"
ULPDR_MIXED_B, Simualtion with device count: 2

/home/adi/Workspace/ns-allinone-3.37/ns-3.37/ns3 run "lora-device-classes-example --gwDc=True --endDeviceType=A --appPeriod=600 --simulationTime=800 --nDevices=5 --dataUpType=2 --realisticModel=False"
ULPDR_MIXED_A, Simualtion with device count: 6
/home/adi/Workspace/ns-allinone-3.37/ns-3.37/ns3 run "lora-device-classes-example --gwDc=False --endDeviceType=B --appPeriod=600 --simulationTime=800 --nDevices=2 --dataUpType=0 --realisticModel=False"ULPDR_UNCONFIRMED_B, Simualtion with device count: 2

/

+0.000000000s Completed configuration
+0.000000000s Running simulation...
Computing performance metrics...


/home/adi/Workspace/ns-allinone-3.37/ns-3.37/ns3 run "lora-device-classes-example --gwDc=False --endDeviceType=B --appPeriod=600 --simulationTime=800 --nDevices=2 --dataUpType=2 --realisticModel=False"
ULPDR_MIXED_B, Simualtion with device count: 2
/home/adi/Workspace/ns-allinone-3.37/ns-3.37/ns3 run "lora-device-classes-example --gwDc=True --endDeviceType=A --appPeriod=600 --simulationTime=800 --nDevices=6 --dataUpType=0 --realisticModel=False"
ULPDR_UNCONFIRMED_A, Simualtion with device count: 7
/home/adi/Workspace/ns-allinone-3.37/ns-3.37/ns3 run "lora-device-classes-example --gwDc=True --endDeviceType=C --appPeriod=600 --simulationTime=800 --nDevices=2 --dataUpType=0 --realisticModel=False"
ULPDR_UNCONFIRMED_C, Simualtion with device count: 2
/home/adi/Workspace/ns-allinone-3.37/ns-3.37/ns3 run "lora-device-classes-example --gwDc=True --endDeviceType=C --appPeriod=600 --simulationTime=800 --nDevices=2 --dataUpType=0 --realisticModel=False"
ULPDR_UNCONFIRMED_C, Simualtion with devic

...

1.0 L: 0.0025
1.0 L: 0.0025
1.0 L: 0.00375
1.0 L: 0.00375
1.0 L: 0.00375
1.0 L: 0.00625
1.0 L: 0.00875
1.0 L: 0.00875
1.0 L: 0.0075
1.0 L: 0.01
1.0 L: 0.01625
1.0 L: 0.02
1.0 L: 0.025
1.0 L: 0.02625
1.0 L: 0.0325
1.0 L: 0.04
1.0 L: 0.0575
1.0 L: 0.06375
1.0 L: 0.085
1.0 L: 0.10375
1.0 L: 0.12
1.0 L: 0.15125
0.9933774834437086 L: 0.18875
0.9949748743718593 L: 0.24875
0.9958847736625515 L: 0.30375
0.9967741935483871 L: 0.3875
1.0 L: 0.45375
0.9844444444444445 L: 0.5625
0.9823008849557522 L: 0.70625
0.9857549857549858 L: 0.8775
0.9742086752637749 L: 1.06625
0.974624060150376 L: 1.33
0.9552012148823082 L: 1.64625
0.9156041287188829 L: 2.05875
0.8720310227823558 L: 2.57875
0.8208955223880597 L: 3.1825
0.7448015122873346 L: 3.9675
0.6517970940606679 L: 4.90375
0.5554174435467164 L: 6.03375
0.46194837635303915 L: 7.50625


Computing performance metrics...


1.0 L: 0.0025
1.0 L: 0.0025
1.0 L: 0.00375
1.0 L: 0.00375
1.0 L: 0.00375
1.0 L: 0.00625
1.0 L: 0.00875
1.0 L: 0.00875
1.0 L: 0.0075
1.0 L: 0.01
1.0 L: 0.01625
1.0 L: 0.02
1.0 L: 0.025
1.0 L: 0.02625
1.0 L: 0.0325
1.0 L: 0.04
1.0 L: 0.0575
1.0 L: 0.06375
1.0 L: 0.085
1.0 L: 0.10375
1.0 L: 0.12
1.0 L: 0.15125
1.0 L: 0.18875
0.9899497487437185 L: 0.24875
1.0 L: 0.30375
0.9967741935483871 L: 0.3875
1.0 L: 0.45375
0.9844444444444445 L: 0.5625
0.9787610619469026 L: 0.70625
0.9857549857549858 L: 0.8775
0.9742086752637749 L: 1.06625
0.9736842105263158 L: 1.33
0.9552012148823082 L: 1.64625
0.9162112932604736 L: 2.05875
0.8715462918080465 L: 2.57875
0.8205027494108406 L: 3.1825
0.7400756143667296 L: 3.9675
0.6571501401988274 L: 4.90375
0.5562461155997513 L: 6.03375
0.46161532056619486 L: 7.50625


Computing performance metrics...


1.0 L: 0.0025
1.0 L: 0.0025
1.0 L: 0.00375
1.0 L: 0.00375
1.0 L: 0.00375
1.0 L: 0.00625
1.0 L: 0.00875
1.0 L: 0.00875
1.0 L: 0.0075
1.0 L: 0.01
1.0 L: 0.01625
1.0 L: 0.02
1.0 L: 0.025
1.0 L: 0.02625
1.0 L: 0.0325
1.0 L: 0.04
1.0 L: 0.0575
1.0 L: 0.06375
1.0 L: 0.085
1.0 L: 0.10375
1.0 L: 0.12
1.0 L: 0.15125
0.9933774834437086 L: 0.18875
0.9949748743718593 L: 0.24875
1.0 L: 0.30375
0.9967741935483871 L: 0.3875
1.0 L: 0.45375
0.9844444444444445 L: 0.5625
0.9823008849557522 L: 0.70625
0.9857549857549858 L: 0.8775
0.9742086752637749 L: 1.06625
0.974624060150376 L: 1.33
0.9552012148823082 L: 1.64625
0.9162112932604736 L: 2.05875
0.8720310227823558 L: 2.57875
0.8208955223880597 L: 3.1825
0.7400756143667296 L: 3.9675
0.6574050471577874 L: 4.90375
0.5574891236793039 L: 6.03375
0.4607826810990841 L: 7.50625
10630.328265190125
Successfully sent email to aditya.jagatha@trojans.dsu.edu:
